Você já criou o banco de dados, já criou as tabelas e inseriu os dados.
Praticamos até aqui a arquitetura de um banco de dados, DDl, DML para criar as tabelas e
inserir, deletar e manipular dados dentro do nosso banco.

# instalações:

In [2]:
pip install psycopg2 pandas

  Using cached pandas-2.2.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached numpy-2.0.1-cp312-cp312-macosx_14_0_arm64.whl.metadata (60 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.2-cp312-cp312-macosx_11_0_arm64.whl (11.3 MB)
Using cached numpy-2.0.1-cp312-cp312-macosx_14_0_arm64.whl (5.0 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import psycopg2
import pandas

#### Se conectar ao banco:

In [4]:
# Conectando ao banco de dados
conn = psycopg2.connect(
    dbname="seu_banco_de_dados",
    user="seu_usuario",
    password="sua_senha",
    host="localhost",  # ou outro host onde seu banco de dados está localizado
    port="5432"
)
cursor = conn.cursor()

#### 1. Selecione todos produtos de uma categoria.

In [5]:
def select_products_by_category(category):
    query = "SELECT * FROM Produtos WHERE Categoria = %s"
    cursor.execute(query, (category,))
    return cursor.fetchall()

#### 2. Nome e endereço dos clientes que fizeram um pedido em 2024

In [10]:
def select_clients_orders_2024():
    query = """
    SELECT c.Nome, c.Endereco
    FROM Clientes c
    JOIN Pedidos p ON c.ID_Cliente = p.ID_Cliente
    WHERE EXTRACT(YEAR FROM p.Data_Pedido) = 2024
    """
    cursor.execute(query)
    return cursor.fetchall()

#### 3. Produtos com preço entre 50 e 100:

In [11]:
def select_products_price_range():
    query = "SELECT * FROM Produtos WHERE Preco BETWEEN 50 AND 100"
    cursor.execute(query)
    return cursor.fetchall()

#### 4. Número total de pedidos feitos em julho de 2024:

In [12]:
def count_orders_july_2024():
    query = """
    SELECT COUNT(*) AS Total_Pedidos
    FROM Pedidos
    WHERE EXTRACT(YEAR FROM Data_Pedido) = 2024
      AND EXTRACT(MONTH FROM Data_Pedido) = 7
    """
    cursor.execute(query)
    return cursor.fetchone()

#### 5. Total pago por cada pedido:

In [13]:
def total_paid_per_order():
    query = "SELECT ID_Pedido, SUM(Valor_Pago) AS Total_Pago FROM Pagamentos GROUP BY ID_Pedido"
    cursor.execute(query)
    return cursor.fetchall()

#### 6. Nome dos clientes e valor total dos seus pedidos

In [14]:
def total_value_per_client():
    query = """
    SELECT c.Nome, SUM(p.Valor_Total) AS Total_Pedido
    FROM Clientes c
    JOIN Pedidos p ON c.ID_Cliente = p.ID_Cliente
    GROUP BY c.Nome
    """
    cursor.execute(query)
    return cursor.fetchall()

#### 7. Produtos com estoque menor que 10:

In [15]:
def products_low_stock():
    query = "SELECT * FROM Produtos WHERE Estoque < 10"
    cursor.execute(query)
    return cursor.fetchall()

#### 8. Pedidos com status e quantidade de itens em cada pedido:

In [16]:
def orders_with_item_count():
    query = """
    SELECT p.ID_Pedido, p.Status, COUNT(i.ID_Item) AS Quantidade_Itens
    FROM Pedidos p
    JOIN Itens_Pedido i ON p.ID_Pedido = i.ID_Pedido
    GROUP BY p.ID_Pedido, p.Status
    """
    cursor.execute(query)
    return cursor.fetchall()

#### 9. Produtos que não foram vendidos em nenhum pedido:

In [17]:
def unsold_products():
    query = """
    SELECT * FROM Produtos
    WHERE ID_Produto NOT IN (
        SELECT DISTINCT ID_Produto
        FROM Itens_Pedido
    )
    """
    cursor.execute(query)
    return cursor.fetchall()

#### 10. Os 10 maiores clientes por quantidade de pedidos:

In [18]:
def top_10_clients_by_orders():
    query = """
    SELECT c.Nome, COUNT(p.ID_Pedido) AS Quantidade_Pedidos
    FROM Clientes c
    JOIN Pedidos p ON c.ID_Cliente = p.ID_Cliente
    GROUP BY c.Nome
    ORDER BY Quantidade_Pedidos DESC
    LIMIT 10
    """
    cursor.execute(query)
    return cursor.fetchall()

#### 11. Produto mais caro e mais barato:

In [20]:
def most_and_least_expensive_products():
    query = """
    SELECT 'Mais Caro' AS Tipo, ID_Produto, Nome, Descricao, Preco, Estoque, Categoria
    FROM (
        SELECT * FROM Produtos
        ORDER BY Preco DESC
        LIMIT 1
    ) AS Mais_Caro

    UNION ALL

    SELECT 'Mais Barato' AS Tipo, ID_Produto, Nome, Descricao, Preco, Estoque, Categoria
    FROM (
        SELECT * FROM Produtos
        ORDER BY Preco ASC
        LIMIT 1
    ) AS Mais_Barato
    """
    cursor.execute(query)
    return cursor.fetchall()

#### 12. Idade média dos clientes:

In [21]:
def average_client_age():
    query = "SELECT AVG(EXTRACT(YEAR FROM AGE(Data_Nascimento))) AS Idade_Media FROM Clientes"
    cursor.execute(query)
    return cursor.fetchone()

#### 13. Clientes que possuem o mesmo sobrenome:

In [22]:
def clients_with_same_last_name():
    query = """
    SELECT Nome, COUNT(*) AS Quantidade
    FROM Clientes
    GROUP BY Nome
    HAVING COUNT(*) > 1
    """
    cursor.execute(query)
    return cursor.fetchall()

#### 14. Pedidos feitos por clientes cujo nome começa com 'A':

In [23]:
def orders_by_clients_name_starts_with_A():
    query = """
    SELECT p.*
    FROM Pedidos p
    JOIN Clientes c ON p.ID_Cliente = c.ID_Cliente
    WHERE c.Nome LIKE 'A%'
    """
    cursor.execute(query)
    return cursor.fetchall()

#### 15. Produtos, sua categoria e indicação de estoque:

In [24]:
def products_with_stock_status():
    query = """
    SELECT Nome, Categoria,
           CASE 
               WHEN Estoque > 0 THEN 'Em Estoque'
               ELSE 'Esgotado'
           END AS Status_Estoque
    FROM Produtos
    """
    cursor.execute(query)
    return cursor.fetchall()

#### Exemplo de chamada de função e exibição dos resultados:

In [25]:
if __name__ == "__main__":
    print(select_products_by_category('Eletrônicos'))
    print(select_clients_orders_2024())
    print(select_products_price_range())
    print(count_orders_july_2024())
    print(total_paid_per_order())
    print(total_value_per_client())
    print(products_low_stock())
    print(orders_with_item_count())
    print(unsold_products())
    print(top_10_clients_by_orders())
    print(most_and_least_expensive_products())
    print(average_client_age())
    print(clients_with_same_last_name())
    print(orders_by_clients_name_starts_with_A())
    print(products_with_stock_status())

[(2, 'key', 'Me effect detail laugh family perhaps anything beyond.', Decimal('276.75'), 90, 'Eletrônicos'), (5, 'institution', 'Already agency trip.', Decimal('220.61'), 84, 'Eletrônicos'), (6, 'bar', 'Teacher beautiful authority rule.', Decimal('609.60'), 71, 'Eletrônicos'), (12, 'identify', 'Per better nor half.', Decimal('580.89'), 16, 'Eletrônicos'), (18, 'impact', 'Daughter join apply lawyer yet address.', Decimal('558.72'), 85, 'Eletrônicos'), (21, 'southern', 'Interview Mr season system newspaper decade remember part.', Decimal('762.80'), 72, 'Eletrônicos'), (22, 'happy', 'Next customer which blood society for.', Decimal('738.21'), 12, 'Eletrônicos'), (28, 'fill', 'Although skill open Republican production student.', Decimal('933.55'), 63, 'Eletrônicos'), (33, 'oil', 'Meet east history sport.', Decimal('272.22'), 24, 'Eletrônicos'), (39, 'especially', 'Entire training better offer discussion.', Decimal('735.93'), 62, 'Eletrônicos'), (40, 'glass', 'Break environmental position.'

#### Fechar a conexão:

In [26]:
cursor.close()
conn.close()